# Simulation of planetary orbits

## Introduction

Developed and executed complex simulation codes in Python using the Euler method to accurately model the 
dynamic behavior of the solar system over a 10-year time span with different initial conditions.
Generated comprehensive data visualisations and analysis reports to effectively communicate the findings of the solar 
system simulation, resulting in increased understanding of planetary motion.

The laws of gravitational attraction are derived from Newtonian Mechanics and implemented ny using a numerical method called Euler method. General Relativity is not taken into account for the sake of simplicity. The vetor force of attraction is given by: 

$$F_{12} = - G \frac{ m_1 m_2}{r_{21}^2} \hat{r_{21}}$$

where

 - $F_{12}$ is the gravitational force exerted on the first mass by the second mass, $\mathrm{N}$
 - $G$ is the gravitation constant, 6.674 × 10−11 $\mathrm{N \ m^2 \ kg^{-2}}$
 - $m_1$ is the mass of the first mass, $\mathrm{kg}$
 - $m_2$ is the mass of the second mass, $\mathrm{kg}$ 
 - $\hat{r_{21}}$ is unit position vector from the second mass to the first mass
 
The position and velocity of a planet is updated using a time step, which will be represented as dt in this notebook, and can be found by using the following equations which utilise a numeric method:

$$ \delta r = v \delta t $$
$$ \delta v = a \delta t $$
$$ r(t + \delta t) = r(t) + v \delta t $$
$$ v(t + \delta t) = v(t) + a \delta t $$

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
import numpy as np
from numpy.linalg import norm

## Part A

### 1. Calculating the gravitational force between two objects

You should complete the following function, without changing its name, arguments or docstring, to calculate the force on one object due to the gravitational field of another.

In [ ]:
def force(pos1, pos2, m1, m2):
    """
    Returns the gravitational force exerted by object 2 on object 1.
    Input:
      - pos1 = position vector of first object  (NumPy array)
      - pos2 = position vector of second object (NumPy array)
      - m1   = mass of first object
      - m2   = mass of second object
    Depends on:
      - G    = gravitational constant (global variable)
    """
    
    r_vector = np.array(pos2) - np.array(pos1)                                                # relative position vector
    magnitude_r = np.linalg.norm(r_vector)                                # magnitude of the position vector
    G_force_magnitude = (G * m1 * m2) / (magnitude_r ** 2)                # magnitude of the gravitational force
    G_force_vector = G_force_magnitude * r_vector / magnitude_r           # vector of the gravitational force
    return G_force_vector

#### Testing your function

The following cell applies some tests to help you make sure your `force` function works correctly before you use it in the rest of the task. You do not need to understand the details of how it works, but it may help you narrow down any bugs in your code. If each line of output starts with `OK` then it is likely (but not guaranteed) that you have implemented the function correctly.

Please leave the code in this cell unchanged: you may add your own tests if you wish, but these should be in a separate cell.

In [ ]:
################################################
#                                              #
# Test force is correct in a few simple cases. #
#                                              #
#   DO NOT CHANGE THE CODE IN THIS CELL.       #
#                                              #
################################################

def test_force(pos1, pos2, m1, m2, expected_force):
    """Check whether force function gives expected results."""
    epsilon = 1e-10
    f = force(np.array(pos1), np.array(pos2), m1, m2)
    if not isinstance(f,np.ndarray):
        print(f"ERROR: function should return a vector but returns {f}.")
        return
    args_as_string = f"({pos1}, {pos2}, {m1}, {m2})"
    error = norm(f-expected_force)
    if error<epsilon:
        print(f"OK: correct results for input {args_as_string}")
    else:
        print(f"ERROR: wrong results for input {args_as_string}")
        print(f"  expected: {expected_force}")
        print(f"  got:      {f}")

# Set global variable G to 1 for now to make testing easier
G=1

# Test force with some simple cases
test_force((0,0),(1,0),1,1,(1,0))    # distance = 1 in x direction
test_force((1,0),(0,0),1,1,(-1,0))   # swap objects
test_force((0,0),(2,0),1,1,(0.25,0)) # distance = 2
test_force((0,0),(0,1),1,1,(0,1))    # distance = 1 in y direction
test_force((10,0),(10,1),1,1,(0,1))  # displaced from origin
test_force((0,0),(1,0),2,1,(2,0))    # non-unit mass 1
test_force((0,0),(1,0),1,2,(2,0))    # non-unit mass 2

### 2. Calculating the motion of a planet in the gravitational field of a star

You should complete the following function, without changing its name, arguments or docstring, to calculate the new position and velocity of a planet after a time step `dt`, in the gravitational field of a star of a given mass situated at the origin. This function will need to call `force` to calculate the acceleration vector of the planet.

In [ ]:
def move_planet(position, velocity, m_star, dt):
    """
    Calculate motion of planet in the gravitational field of a star with given mass
    at the origin, using Euler's method.
    
    Input:
      - position: position vector of planet at start of time step (NumPy array)
      - velocity: velocity vector of planet at start of time step (NumPy array)
      - m_star:   mass of star
      - dt:       time step
      
    Output: (position_new, velocity_new)
      - position_new: position vector of planet at end of time step (NumPy array)
      - velocity_new: velocity vector of planet at end of time step (NumPy array)
      
    Depends on:
      - force = function to calculate the gravitational force between two objects
    """
    
    p_initial = np.array(position)
    v = np.array(velocity)
    position_new = p_initial + v * dt
    m_planet = 1   # Take any number. It does not matter because it will be cancelled in the next step
    acceleration = force(np.array([0,0]), position, m_star, m_planet) / m_planet
    velocity_new = v - acceleration * dt
    return position_new, velocity_new

#### Testing your function

The following cell applies some tests to help you make sure your `move_planet` function works correctly before you use it in the rest of the task. You do not need to understand the details of how it works, but it may help you narrow down any bugs in your code. If each line of output starts with `OK` then it is likely (but not guaranteed) that you have implemented the function correctly.

Please leave the code in this cell unchanged: you may add your own tests if you wish, but these should be in a separate cell. 

In [ ]:
######################################################
#                                                    #
# Test move_planet is correct in a few simple cases. #
#                                                    #
#   DO NOT CHANGE THE CODE IN THIS CELL.             #
#                                                    #
######################################################

def test_move_planet(position, velocity, m_star, dt, expected_pos, expected_vel):
    """Check whether move_planet function gives expected results."""
    epsilon = 1e-10
    results = move_planet(position, velocity, m_star, dt)
    if not isinstance(results, tuple):
        print(f"ERROR: function should return two vectors but returns {results}.")
        return
    if not len(results)==2:
        print(f"ERROR: function should return two vectors but returns {results}.")
        return
    pos, vel = results
    if not (isinstance(pos,np.ndarray) and isinstance(vel,np.ndarray)):
        print(f"ERROR: function should return two vectors but returns {results}.")
        return
    args_as_string = f"{position}, {velocity}, {m_star}, {dt}"
    err_pos, err_vel = norm(pos - expected_pos), norm(vel - expected_vel)
    if err_pos < epsilon and err_vel < epsilon:
        print(f"OK: correct results for input {args_as_string}")
    else:
        print(f"ERROR: wrong results for input {args_as_string}")
        print(f"  expected: {expected_pos, expected_vel}")
        print(f"  got:      {results}")

# Set global variable G to 1 for now to make testing easier
G=1

# Test move_planet with some simple cases
test_move_planet((1,0), (1,0), 1, 0, (1,0), (1,0))    # dt = 0: output = input
test_move_planet((1,0), (1,0), 1, 1, (2,0), (0,0))    # moving away from star
test_move_planet((0,1), (0,-1), 1, 1, (0,0), (0,-2))  # moving towards star
test_move_planet((1,0), (0,1), 1, 1, (1,1), (-1,1))   # moving past star
test_move_planet((1,0), (0,1), 1, 0.1, (1,0.1), (-0.1,1))  # smaller dt
test_move_planet((1,0), (0,1), 2, 0.1, (1,0.1), (-0.2,1))  # larger star mass
test_move_planet((2,0), (1,0), 1, 1, (3,0), (0.75,0)) # non-unit distance

### 3. Calculating the orbit of a planet

You should complete the following function, without changing its name, arguments or docstring, to calculate the $x$ and $y$ coordinates of a planet at each time step.

In [ ]:
def trajectory(position, velocity, m_star, dt, t_max):
    """
    Calculate trajectory of planet from given starting position, using Euler's method.
    
    Input:
      - position: position vector of planet at start of simulation [m] (NumPy array)
      - velocity: velocity vector of planet at start of simulation [m] (NumPy array)
      - m_star:   mass of star [kg]
      - dt:       time step    [s]
      - t_max:    duration of calculated motion [s]

    Output: (x_arr, y_arr) [m]
        where x_arr and y_arr are NumPy arrays containing the x and y coordinates of
        the planet at each time step, starting with the initial position
    """

    p_start_sim = np.array([0., 0.])                           
    t = 0
    x, y = position
    x_arr, y_arr = [x], [y]
    while t<=t_max:
        t += dt
        position, velocity =  move_planet(position, velocity, m_star, dt)
        x, y = position
        x_arr.append(x)
        y_arr.append(y)
    return x_arr, y_arr

#### Testing your function

If you have implemented all three functions correctly, the following cell should show an almost circular orbit.

In [ ]:
G       = 6.6743e-11 # gravitational constant [m^3 kg^-1 s^-2]
YEAR    = 3.154e7    # one year in seconds
M_STAR  = 2.5e30     # mass of star [kg]

dt = YEAR/10000      # time step for Euler's method
pos = (2.0e11, 0)    # initial position (x,y) [m]
vel = (0,2.89e4)     # initial velocity (vx,vy) [m/s]

# Calculate trajectory
x_test, y_test = trajectory(pos,vel,M_STAR,dt,2*YEAR)

# Plot trajectory
fig, axes = plt.subplots()
axes.set_aspect('equal')
plt.title('Trajectory of planet')
plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.plot(x_test, y_test)

### 4. Animating the orbit

To better visualise the motion of the planet, we create an animation showing the changing position of the planet, with a trail to show its path.

As seen in unit 9, we define a function that updates the display of the planet and trail:-

In [ ]:
def animate_with_trail(i, planet, trail, x_arr, y_arr):
    """
    Update display for projectile motion, with trail.
    
    Arguments:
        i:      frame number (from 0 at time = 0)
        planet: Line2D object containing coordinates of planet
        trail:  Line2D object containing coordinates of trail
        x_arr:  array of x coordinate at teach time step
        y_arr:  array of y coordinate at teach time step

    Result: updates coordinates in Line2D objects provided as arguments
    "bodies" and "trails".
    """
    x, y = x_arr[i], y_arr[i]
    planet.set_data([x],[y])                # Body gets coordinates of current position
    trail.set_data(x_arr[:i+1],y_arr[:i+1]) # Trail has all points up to the current one

To make it easier to create different simulations without repeating too much code, we define a function to create an animation using the calculated trajectory. As well as the arrays of x and y values, we have to pass it the time step used in the simulation, the number of frames per second to use in the animation, and the "speedup" factor, which is the ratio of the simulated time (which is typically of order years) to the real time (typically of order seconds).

To create a relatively smoothly animation with the finite power of our computer, we can't plot every position we have calculated along the trajectory, but have to pick every $N$th position, where $N$ is chosen to match our animation frame rate to our calculated trajectory.

In [ ]:
def create_animation(x_arr, y_arr, dt, fps, speedup):
    """
    Create an animation of an object with given trajectory.
    
    Arguments:
        x_arr, y_arr: arrays of x and y coordinates at each time step [m]
        dt:           time step in simulation [s]
        fps:          number of frames per second for animation
        speedup:      ratio of simulated time to screen time
    """
    # Find range of coordinates to include full trajectory
    x_max = max(np.abs(x_arr))
    y_max = max(np.abs(y_arr))
    r_max = 1.05 * max(x_max, y_max)
    
    # Create and configure figure and axes
    plt.ioff()
    fig, axes = plt.subplots()
    axes.set_xlim(-r_max,r_max)
    axes.set_ylim(-r_max,r_max)
    axes.set_aspect('equal')
    axes.set_title('Trajectory of planet')
    axes.set_xlabel('x [m]')
    axes.set_ylabel('y [m]')
    plt.plot(0,0,'o',color="orange", markersize=10)

    # Create Line2D objects to represent body and trail
    planet,  = axes.plot([],[],'o')
    trail,   = axes.plot([],[],'-')

    # Get arrays of coordinates to display (subset of provided data)
    frame_interval = 1/fps                  # time between frames [s]
    skip = round(frame_interval*speedup/dt) # number of calculated points per frame
    x_plot = x_arr[::skip]  # take every Nth value (N = skip)
    y_plot = y_arr[::skip]
    n_frames = len(x_plot)

    # Create and return animation object
    ani = animation.FuncAnimation(fig,animate_with_trail, frames=n_frames, interval=frame_interval*1000,
                                  fargs=[planet, trail, x_plot, y_plot])
    return ani

***Animating your calculated orbit.***

In the cell below, write some code that calls this function to create an animation of the orbit you have calculated above. You should choose parameters to create a fairly smooth animation that takes a few seconds to run.

In [ ]:
G       = 6.6743e-11 # gravitational constant [m^3 kg^-1 s^-2]
YEAR    = 3.154e7    # one year in seconds
M_STAR  = 2.5e30     # mass of star [kg]

dt = YEAR/10000      # time step for Euler's method
pos = (2.0e11, 0)    # initial position (x,y) [m]
vel = (0,2.89e4)     # initial velocity (vx,vy) [m/s]


fps = 60
speedup = 5000000

In [ ]:
ani_orbit = create_animation(x_test, y_test, dt, fps, speedup)

HTML(ani_orbit.to_jshtml())

### 5. Investigation

You should insert code and text cells below as required to investigate and discuss the effect of changing the parameters of the animation: time step, mass of star, initial position and velocity of planet.

### *changing time-step*

In [ ]:
# This dt is 10 times larger than the one used in section 3

x_test, y_test = trajectory(pos,vel,M_STAR,YEAR/1000,2*YEAR)

ani_orbit_1 = create_animation(x_test, y_test, YEAR/1000, fps, speedup)
HTML(ani_orbit_1.to_jshtml())

In [ ]:
# This dt is 10 times smaller than the one used in section 3

x_test, y_test = trajectory(pos,vel,M_STAR,YEAR/100000,2*YEAR)
    
ani_orbit_2 = create_animation(x_test, y_test, YEAR/100000, fps, speedup)
HTML(ani_orbit_2.to_jshtml())

##### Effect of changing dt
The motion of the planet with differen time steps (10 times and 0.1 times) has been noticed and a higher time step leading to growing radius of orbit with respect of passage of time is observed. With the timestep updated to 0.1 dt, the change is radius after a single orbit is negligible, on the other hand, a clear and noticeable increase in radius can be seen when timestep is magnified by 10 times the original dt.

### *changing the mass of the star*

In [ ]:
# This mass of the star is 2 times larger than the one used in section 3

x_test, y_test = trajectory(pos,vel,5e30,dt,2*YEAR)

ani_orbit_3 = create_animation(x_test, y_test, dt, fps, speedup)
HTML(ani_orbit_3.to_jshtml())

In [ ]:
# This mass of the star in 2 times smaller than the one used in section 3

x_test, y_test = trajectory(pos,vel,1.25e30,dt,2*YEAR)

ani_orbit_4 = create_animation(x_test, y_test, dt, fps, speedup)
HTML(ani_orbit_4.to_jshtml())

###### Effect of changing mass of the star

Motion of the planet with different mass of the star is observed.




When the mass of the star is increased to twice its original mass, the gravitational force between the star and planet is increased which resulted in reduced radius during the first half of the orbit. The orbit makes a shape of ellipse with eccentricity < 1. Another visible feature of the orbit is the increasing radius with each elapsed revolution around the star, which is most noticeable at the aphelion.

When the mass of the star is reduced by two times, the gravitational attraction between the star and the planet gets reduced and the planet starts to move away from the star in a hyperbolic trajectory with eccentricity > 1. As the planet continues it trajectory, it gets farther and farther from the star and the attractive force weekens by F $\propto$ $\frac{1}{r^{2}}$, leading to lesser and lesser curved path. 

### *changing the initial position of the planet*

In [ ]:
# The initial position of the planet is 2 times further

x_test, y_test = trajectory((4e11, 0),vel,M_STAR,dt,2*YEAR)

ani_orbit_5 = create_animation(x_test, y_test, dt, fps, speedup)
HTML(ani_orbit_5.to_jshtml())

In [ ]:
# The initial position of the planet 2 times closer

x_test, y_test = trajectory((1e11, 0),vel,M_STAR,dt,2*YEAR)

ani_orbit_6 = create_animation(x_test, y_test, dt, fps, speedup)
HTML(ani_orbit_6.to_jshtml())

###### Effects of changing initial position of the planet

The motion of the planet with different initial positions relative to the star is observed.

When the planet was placed at $\frac{1}{2}$ of the original x-coordinate, the orbit exhibits the shape of an ecclipse with eccentricity < 1. The size of the orbit increases with each revolution. The velocity of the planet changes with respect to Kepler's second law. 

It behaves the same way as when the mass of the star doubled.

When the planet moves away by a factor of $2$ relative to the initial x-coordinate, the gravitational force weekens and the planet makes its trajectory away from the star in a hyperbolic curve with an eccentricity of > 1. 

It behaves the same way as when the mass of the star halved.

### *changing the velocity of the planet*

In [ ]:
# Increasing the initial velocity of the planet 2 times

x_test, y_test = trajectory(pos,(0,5.78e4),M_STAR,dt,2*YEAR)

ani_orbit_7 = create_animation(x_test, y_test, dt, fps, speedup)
HTML(ani_orbit_7.to_jshtml())

In [ ]:
# Decreasing the initial velocity of the planet 2 times

x_test, y_test = trajectory(pos,(0,1.445e4),M_STAR,dt,2*YEAR)

ani_orbit_8 = create_animation(x_test, y_test, dt, fps, speedup)
HTML(ani_orbit_8.to_jshtml())

##### Effects of changing the initial velocity of the planet

The motion of the planet with different initial volcities is observed.

When the velocity of the planet is increased by $10$ times, it pushes the planets out of the stronger regions of the gravitational field of attraction and the planet, no longer bounds by the force, follows a linear trajectory.

When the velocity of the planet is halved, the planet can not make a full circle around the planet and is drawn into the orbit to follow an elliptical shape, with each radi of the orbit greater than the last. 

## Part B

### Two non-interacting planets

#### Animation of two planets orbiting a single star

In [ ]:
def move_planets(position1, velocity1, position2, velocity2, m_star, dt):
    """
    Calculate motion of planets in the gravitational field of a star with given mass
    at the origin, using Euler's method.
    
    Input:
      - position1: position vector of planet1 at start of time step (NumPy array)
      - velocity1: velocity vector of planet1 at start of time step (NumPy array)
      - position2: position vector of planet2 at start of time step (NumPy array)
      - velocity2: velocity vector of planet2 at start of time step (NumPy array)
      - m_star:    mass of star
      - dt:        time step
      
    Output: (position_new1, velocity_new1, position_new2, velocity_new2)
      - position_new1: position vector of planet1 at end of time step (NumPy array)
      - velocity_new1: velocity vector of planet1 at end of time step (NumPy array)
      - position_new2: position vector of planet2 at end of time step (NumPy array)
      - velocity_new2: velocity vector of planet2 at end of time step (NumPy array)
      
    Depends on:
      - force = function to calculate the gravitational force between two objects
    """
    
    p_initial1 = np.array(position1)
    p_initial2 = np.array(position2)
    v1 = np.array(velocity1)
    v2 = np.array(velocity2)
    
    position_new1 = p_initial1 + v1 * dt
    position_new2 = p_initial2 + v2 * dt
    
    m_planet1 = 1   # Take any number. It does not matter because it will be cancelled in the next step
    m_planet2 = 1
    acceleration1 = force (np.array([0,0]), position1, m_star, m_planet1) / m_planet1
    acceleration2 = force (np.array([0,0]), position2 , m_star, m_planet2) / m_planet2
    velocity_new1 = v1 - acceleration1 * dt
    velocity_new2 = v2 - acceleration2 * dt
    return position_new1, velocity_new1, position_new2, velocity_new2

In [ ]:
def trajectorys(position1,velocity1, position2, velocity2, m_star, dt, t_max):
    """
    Calculate trajectories of planeta from given starting positions, using Euler's method.
    
    Input:
      - position1: position vector of planet1 at start of simulation [m] (NumPy array)
      - velocity1: velocity vector of planet1 at start of simulation [m] (NumPy array)
      - position2: position vector of planet2 at start of simulation [m] (NumPy array)
      - velocity2: velocity vector of planet2 at start of simulation [m] (NumPy array)
      - m_star:   mass of star [kg]
      - dt:       time step    [s]
      - t_max:    duration of calculated motion [s]

    Output: (x_arr1, y_arr1, x_arr2, y_arr2) [m]
        where x_arr and y_arr are NumPy arrays containing the x and y coordinates of
        the planets at each time step, starting with the initial positions
    """

    p_start_sim1 = np.array([0., 0.]) 
    p_start_sim2 = np.array([0., 0.])
    t = 0
    x1, y1 = position1
    x2, y2 = position2
    x_arr1, y_arr1 = [x1], [y1]
    x_arr2, y_arr2 = [x2], [y2]
    while t<=t_max:
        t += dt
        position1, velocity1, position2, velocity2 =  move_planets(position1, velocity1, position2, velocity2, m_star, dt)
        x1, y1 = position1
        x2, y2 = position2
        x_arr1.append(x1)
        y_arr1.append(y1)
        x_arr2.append(x2)
        y_arr2.append(y2)
        
    return x_arr1, y_arr1, x_arr2, y_arr2

In [ ]:
G       = 6.6743e-11 # gravitational constant [m^3 kg^-1 s^-2]
YEAR    = 3.154e7    # one year in seconds
M_STAR  = 2.5e30     # mass of star [kg]

dt = YEAR/10000          # time step for Euler's method
pos1 = (2.0e11, 0)       # initial position of planet1 (x,y) [m]
vel1 = (0,2.89e4)        # initial velocity of planet1 (vx,vy) [m/s]
pos2 = (1.56e11,1.56e11) # initial position of planet2 (x,y) [m]
vel2 = (-1.95e4,1.95e4)  # initial velocity of planet2 (vx,vy) [m/s]

# Calculate trajectories
x_test1, y_test1, x_test2, y_test2 = trajectorys(pos1,vel1,pos2,vel2,M_STAR,dt,2*YEAR)

# Plot trajectories
fig, axes = plt.subplots()
axes.set_aspect('equal')

plt.title('Trajectory of planet')
plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.plot(x_test1, y_test1,x_test2, y_test2)

In [ ]:
def animate_with_trails(i, planet1, trail1, x_arr1, y_arr1, planet2, trail2, x_arr2, y_arr2):
    """
    Update display for projectile motion, with trail.
    
    Arguments:
        i:      frame number (from 0 at time = 0)
        planet: Line2D object containing coordinates of planet
        trail:  Line2D object containing coordinates of trail
        x_arr:  array of x coordinate at teach time step
        y_arr:  array of y coordinate at teach time step

    Result: updates coordinates in Line2D objects provided as arguments
    "bodies" and "trails".
    """
    x1, y1, x2, y2 = x_arr1[i], y_arr1[i], x_arr2[i], y_arr2[i]
    planet1.set_data([x1],[y1])                # Body gets coordinates of current position
    planet2.set_data([x2],[y2])
    trail1.set_data(x_arr1[:i+1],y_arr1[:i+1]) # Trail has all points up to the current one
    trail2.set_data(x_arr2[:i+1], y_arr2[:i+1])

In [ ]:
def create_animations(x_arr1, y_arr1, x_arr2, y_arr2, dt, fps, speedup):
    """
    Create an animation of an object with given trajectory.
    
    Arguments:
        x_arr, y_arr: arrays of x and y coordinates at each time step [m]
        dt:           time step in simulation [s]
        fps:          number of frames per second for animation
        speedup:      ratio of simulated time to screen time
    """
    # Find range of coordinates to include full trajectory
    x_max1 = max(np.abs(x_arr1))
    y_max1 = max(np.abs(y_arr1))
    x_max2 = max(np.abs(x_arr2))
    y_max2 = max(np.abs(y_arr2))
    r_max1 = 1.05 * max(x_max1, y_max1)
    r_max2 = 1.05 * max(x_max2, y_max2)
    
    # Create and configure figure and axes
    plt.ioff()
    fig, axes = plt.subplots()
    axes.set_xlim(-r_max1,r_max1)
    axes.set_ylim(-r_max1,r_max1)
    axes.set_aspect('equal')
    axes.set_title('Trajectory of planet')
    axes.set_xlabel('x [m]')
    axes.set_ylabel('y [m]')
    plt.plot(0,0,'o',color="orange", markersize=10)

    # Create Line2D objects to represent body and trail
    planet1,  = axes.plot([],[],'o')
    trail1,   = axes.plot([],[],'-')
    
    planet2,  = axes.plot([],[],'o')
    trail2,   = axes.plot([],[],'-')

    # Get arrays of coordinates to display (subset of provided data)
    frame_interval = 1/fps                  # time between frames [s]
    skip = round(frame_interval*speedup/dt) # number of calculated points per frame
    x_plot1 = x_arr1[::skip]  # take every Nth value (N = skip)
    y_plot1 = y_arr1[::skip]
    
    x_plot2 = x_arr2[::skip]  # take every Nth value (N = skip)
    y_plot2 = y_arr2[::skip]
    
    n_frames1 = len(x_plot1)
    n_frames2 = len(x_plot2)

  # Create and return animation object
    anis = animation.FuncAnimation(fig,animate_with_trails, frames=n_frames2, interval=frame_interval*1000,
                                  fargs=[planet1, trail1, x_plot1, y_plot1, planet2, trail2, x_plot2, y_plot2])
    return anis

In [ ]:
ani_orbits = create_animations(x_test1, y_test1, x_test2, y_test2, dt, fps, speedup)

HTML(ani_orbits.to_jshtml())

### Two planets with gravitational interactions 

In [ ]:
def move_planets_g(m1, position1, velocity1, m2, position2, velocity2, m_star, dt):
    """
    Calculate motion of planet in the gravitational field of a star with given mass
    at the origin, using Euler's method.
    
    Input:
      - position: position vector of planet at start of time step (NumPy array)
      - velocity: velocity vector of planet at start of time step (NumPy array)
      - m_star:   mass of star
      - dt:       time step
      
    Output: (position_new, velocity_new)
      - position_new: position vector of planet at end of time step (NumPy array)
      - velocity_new: velocity vector of planet at end of time step (NumPy array)
      
    Depends on:
      - force = function to calculate the gravitational force between two objects
    """
    
    pos1 = np.array(position1)
    pos2 = np.array(position2)
    vel1 = np.array(velocity1)
    vel2 = np.array(velocity2)
    
    
    
    #m_planet1 = 6e24   # Take any number. It does not matter because it will be cancelled in the next step
    #m_planet2 = 6e24
    acceleration1 = force (np.array([0,0]), position1, m_star, m1) / m1
    acceleration2 = force (np.array([0,0]), position2, m_star, m2) / m2
    acceleration21 = force (pos1, pos2, m1, m2 ) / m1
    acceleration12 = force (pos1, pos2, m1, m2) / m2
    
    position_new1 = pos1 + vel1 * dt
    position_new2 = pos2 + vel2 * dt
    
    velocity_new1 = vel1 - (acceleration1 + acceleration21) * dt
    velocity_new2 = vel2 - (acceleration2 + acceleration12) * dt
   
   # velocity_new1 = v1 - acceleration1 * dt + v1 - acceleration21 * dt
    #velocity_new2 = v2 - acceleration2 * dt + v1 - acceleration12 * dt
    return position_new1, velocity_new1, position_new2, velocity_new2

In [ ]:
def trajectorys_g(m1, pos1, vel1, m2, pos2, vel2, m_star, dt, t_max):
    """
    Calculate trajectory of planet from given starting position, using Euler's method.
    
    Input:
      - position: position vector of planet at start of simulation [m] (NumPy array)
      - velocity: velocity vector of planet at start of simulation [m] (NumPy array)
      - m_star:   mass of star [kg]
      - dt:       time step    [s]
      - t_max:    duration of calculated motion [s]

    Output: (x_arr, y_arr) [m]
        where x_arr and y_arr are NumPy arrays containing the x and y coordinates of
        the planet at each time step, starting with the initial position
    """

    p_start_sim1 = np.array([0., 0.]) 
    p_start_sim2 = np.array([0., 0.])
    t = 0
    x1, y1 = pos1
    x2, y2 = pos2
    x_arr1, y_arr1 = [x1], [y1]
    x_arr2, y_arr2 = [x2], [y2]
    while t<=t_max:
        t += dt
        pos1, vel1, pos2, vel2 =  move_planets_g(m1, pos1, vel1, m2, pos2, vel2, m_star, dt)
        x1, y1 = pos1
        x2, y2 = pos2
        x_arr1.append(x1)
        y_arr1.append(y1)
        x_arr2.append(x2)
        y_arr2.append(y2)
        
    return x_arr1, y_arr1, x_arr2, y_arr2

In [ ]:
def create_animations_g(m_1, pos_1, vel_1, m_2, pos_2, vel_2, m_star, dt, t_max, fps, speedup):
    """
    Create an animation of an object with given trajectory.
    
    Arguments:
        x_arr, y_arr: arrays of x and y coordinates at each time step [m]
        dt:           time step in simulation [s]
        fps:          number of frames per second for animation
        speedup:      ratio of simulated time to screen time
    """
    #make into numpy arrays for use in calculation
    pos_1 = np.array(pos1)       
    vel_1 = np.array(vel1)
    pos_2 = np.array(pos2)
    vel_2 = np.array(vel2)

    # Calculate trajectories
    x_1, y_1, x_2, y_2 = trajectorys_g(m_1, pos_1, vel_1, m_2, pos_2, vel_2, m_star, dt, t_max)

    #combine trajectories into list for trajectories
    traj_m1 = [x_1, y_1]
    traj_m2 = [x_2, y_2]
    traj_m1_m2 = [traj_m1, traj_m2]
    
    anis_g = create_animations(x_1, y_1, x_2, y_2, dt, fps, speedup)
    return HTML(anis_g.to_jshtml())

In [ ]:
YEAR = 3.154e7

In [ ]:
create_animations_g(6e24, [2.0e11,0], [0,2.89e4], 6e24, [1.56e11,1.56e11], [-1.95e4,1.95e4], 2.5e30, YEAR/10000, 5*YEAR, 50, YEAR/7)

In [ ]:
# m2 = 1000 m1
create_animations_g(6e24, [2.0e11,0], [0,2.89e4], 6e27, [1.56e11,1.56e11], [-1.95e4,1.95e4], 2.5e30, YEAR/10000, 5*YEAR, 10, YEAR/7)

In [ ]:
# m_star is now = 5 times that of given parameter
create_animations_g(6e24, [2.0e11,0], [0,2.89e4], 6e24, [1.56e11,1.56e11], [-1.95e4,1.95e4], 12.5e30, YEAR/10000, 5*YEAR, 50, YEAR/7)

### Solar System

In [ ]:
def animate_multiple(i, planets, trails, trajectories):
    """
    Update display for motion of multiple bodies with trails.
    
    Arguments:
        i:       frame number (from 0 at time = 0)
        planets: list of Line2D objects containing coordinates of each body to move
        trails:  list of Line2D objects containing coordinates of trails
        trajectories: list of trajectories, each of the form [x_arr, y_arr]
                 where x_arr and y_arr are arrays of x and y coordinates at
                 each time step
    
    Note that all trajectories must have the same number of points.
    """
    for j in range(len(trajectories)):    # for each trajectory ...
        trajectory = trajectories[j]
        x_arr, y_arr = trajectory         # get arrays of x and y coordinats
        x, y = x_arr[i], y_arr[i]
        planets[j].set_data([x],[y])
        trails[j].set_data(x_arr[:i+1],y_arr[:i+1]) # Trail has all points up to this one

#[1]         

In [ ]:
def get_bodies_and_trails(axes, n):
    """
    Get the required number of Line2D objects to represent the bodies
    being animated, and the trails left behind them.
    
    Arguments:
        axes: the axes to be used for plotting the bodies
        n:    number of bodies in simulation
    
    Returns: (bodies, trails) where
        bodies is a list of Line2D objects to be plotted as discs
        trails is a list of Line2D objects to be plotted as lines
    """
    bodies = []
    trails = []
    for i in range(n):              # repeat n times
        body,  = axes.plot([],[],'o')           # Line2D with disc markers
        col    = body.get_color()               # get colour of body ...
        trail, = axes.plot([],[],'-',color=col) # and set trail (line) to same colour
        bodies.append(body)         # add Line2D objects to lists
        trails.append(trail)
    return bodies, trails

#[1] 

In [ ]:
def move_body(body,dt):
    current_force = np.array([0.0,0.0])
    
    for other_body in all_bodies:
        if (body["position"][0] != other_body["position"][0]) and (body["position"][1] != other_body["position"][1]):
              current_force += force(body["position"], other_body["position"], body["mass"], other_body["mass"])
    
    acceleration = current_force/body["mass"] # acceleration due to the star's gravity
    velocity_new = np.array(body["velocity"]) + acceleration * dt  # update the velocity
    position_new = np.array(body["position"]) + np.array(body["velocity"]) * dt  # update the position
    return position_new, velocity_new

In [ ]:
def trajectory_body(body, dt, t_max):
    """
    Calculate trajectory of planet from given starting position, using Euler's method.
    
    Input:
      - body:     dictionary containing position, velocity and mass of a body
      - dt:       time step    [s]
      - t_max:    duration of calculated motion [s]

    Output: (x_arr, y_arr) [m]
        where x_arr and y_arr are NumPy arrays containing the x and y coordinates of
        the planet at each time step, starting with the initial position
    """
    ##################
    x_arr = [body["position"][0]]
    y_arr = [body["position"][1]]
    for t in np.arange(0, t_max+dt, dt):
        body["position"], body["velocity"] = move_body(body,dt)
        x_arr.append(body["position"][0])
        y_arr.append(body["position"][1])
    return x_arr, y_arr
    ##################

In [ ]:
#same as creat_animation_multiple but with the number of bodies 
#and trails increased


def create_animation_bodies(trajectories, dt, fps, speedup):
    """
    Create an animation of objects with given trajectories.
    
    Arguments:
        trajectories: list of trajectories, each of the form [x_arr, y_arr]
                 where x_arr and y_arr are arrays of x and y coordinates at
                 each time step
        dt:           time step in simulation [s]
        fps:          number of frames per second for animation
        speedup:      ratio of simulated time to screen time
    """
    # Find range of coordinates to include full trajectories 
    r_max = 1.05 * np.max(np.abs(trajectories))

    # Create and configure figure and axes
    plt.ioff()
    fig, axes = plt.subplots()
    axes.set_xlim(-r_max,r_max)
    axes.set_ylim(-r_max,r_max)
    axes.set_aspect('equal')
    plt.plot(0,0,'o',color="orange", markersize=10)

    # Create Line2D objects to represent bodies and trails
    bodies, trails = get_bodies_and_trails(axes, len(trajectories)) 
    body = bodies[0]
    trail = trails[0]

    # Get arrays of coordinates to display (subset of provided data)
    frame_interval = 1/fps                  # time between frames [s]
    skip = round(frame_interval*speedup/dt) # number of calculated points per frame
    plot_trajectories = []
    for trajectory in trajectories:
        x_arr, y_arr = trajectory
        x_plot = x_arr[::skip]  # take every Nth value (N = skip)
        y_plot = y_arr[::skip]
        plot_trajectory = [x_plot, y_plot]
        plot_trajectories.append(plot_trajectory)
        n_frames = len(x_plot)
    
    # Create and return animation object
    ani = animation.FuncAnimation(fig,animate_multiple, frames=n_frames, interval=frame_interval*1000,
                                  fargs=[bodies, trails, plot_trajectories])
    return ani

In [ ]:
def plot_multiple_bodies(bodies,n_years,dt):
    YEAR = 3.154e7
    t_max = YEAR*n_years
    dt =  YEAR/10000
    
    traj_bodies = []
    for body in bodies:
        x_4, y_4 = trajectory_body(body,dt,t_max)
        traj = [x_4,y_4] 
        traj_bodies.append(traj)

    # One year is represented by 2 seconds in the animation
    speedup = YEAR/2

    # Create and display the animation
    ani = create_animation_bodies(traj_bodies, dt, 50, speedup)
    
    return HTML(ani.to_jshtml())   

In [ ]:
#dictionaries for positions, velocities and masses of bodies
Mercury = {"position":[5.79e10,0],  "velocity":[0,5.37e4], "mass":3.30e23}
Venus   = {"position":[1.082e11,0], "velocity":[0,3.96e4], "mass":4.87e24}
Earth   = {"position":[1.496e11,0], "velocity":[0,3.38e4], "mass":5.97e24}
Mars    = {"position":[2.280e11,0], "velocity":[0,2.73e4], "mass":6.42e23}
Jupiter = {"position":[7.785e11,0], "velocity":[0,1.48e4], "mass":1.898e27}
star    = {"position":[0,0],        "velocity":[0,0],      "mass":2.5e30}

all_bodies = [Mercury, Venus, Earth, Mars, star]

In [ ]:
G=6.67e-11
plot_multiple_bodies(all_bodies,4,3.154e3)

#### The above simulation takes real data as input and therefore models the dynamics of Solar System

#### Note: The codes used in this notebook are mine (except Part A's). I used an alternate method to what was provided for help in the Supplementary notebook. 

## References

[1] Planetary fact sheet-metric. Dr. David R Williams. NASA.

[2] Which planet orbits our sun the fastest? Jeff Mangum. National Radio Astronomy Observatory.
    
[3] Waugh B. PHAS0007 Computing final assignment: supplementary notebook.

[4] Waugh B, Chislett R, Dash L. PHAS0007 Computing Final Assignment 2022-23: Simulating Planetary Orbits.